In [1]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable

userhome: String = "/Users/shawnmeier"
jniPath: String = "/Users/shawnmeier/software/z3/build"
newPath: Array[String] = Array(
  "/Users/shawnmeier/software/z3/build",
  "/Users/shawnmeier/Library/Java/Extensions",
  "/Library/Java/Extensions",
  "/Network/Library/Java/Extensions",
  "/System/Library/Java/Extensions",
  "/usr/lib/java",
  "."
)
res0_3: String = "/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:."
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, 

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.ExpTag
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.SpecSpace
import edu.colorado.plv.bounder.lifestate.SpecSignatures
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull}
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData

Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File(System.getProperty("user.dir"))

java.library.path set to: /Users/shawnmeier/software/z3/build:/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:.


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.ExpTag

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import edu.colorado.plv.bounder.PickleSpec

import edu.colorado.plv.bounder.lifestate.SpecSpace

import edu.colorado.plv.bounder.lifestate.SpecSignatures

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull}

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await

import almond.interpreter.api.DisplayData


android_home_possible: List[String] = List(
  "/Users/shawnmeier/Library/Android/sdk",
  "/Users/shawnmeier/Android/Sdk"
)
android_home: String = "/Users/shawnmeier/Library/Android/sdk"
res1_26: String = "/Users/shawnmeier/Documents/source/bounder/notebooks/ossExp"
expDir: File = /Users/shawnmeier/Documents/source/bounder/notebooks/ossExp/SpecGen

In [3]:
val db = new ExperimentsDb(Some(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}

Displayers.register(classOf[Seq[TableAble]], (people: Seq[TableAble]) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(people.head.headers.map(v => th(v))),
        for (row <- people) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

Initializing database


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@718a33be
defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined trait TableAble

In [4]:
case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) extends TableAble{
    override def toString = s"JoinedResultRow(summary = ${summary}, apk = ${apk}"
    def getSummary:BounderUtil.ResultSummary = read[BounderUtil.ResultSummary](summary)
    val getConfig:RunConfig = read[RunConfig](config)
    val getInitQuery = read[InitialQuery](qry)
    assert(getConfig.initialQuery.exists{q => q == getInitQuery})
    def headers = List("tag", "summary", "qry", "apk")
    def values = List (tag  ,  summary , qry  , apk)
    def merge(other:JoinedResultRow):JoinedResultRow = {
        assert(tag == other.tag)
        assert(qry == other.qry)
        assert(config == other.config)
        assert(apk == other.apk)
        
        JoinedResultRow(tag, write[BounderUtil.ResultSummary](Driver.reduceResults(getSummary, other.getSummary)), loc ++ other.loc,qry, config, apk)
    }
}

case class SummaryRow(witnessed:Set[JoinedResultRow] = Set.empty, unreachable:Set[JoinedResultRow] = Set.empty, 
                      timeout:Set[JoinedResultRow] = Set.empty, err:Set[JoinedResultRow] = Set.empty) extends TableAble{
    def headers =  List("witnessed", "unreachable", "timeout", "err")
    def values = List(witnessed.size, unreachable.size, timeout.size, err.size)
    def add(row:JoinedResultRow):SummaryRow = {
        row.getSummary match{
            case BounderUtil.Timeout => this.copy(timeout = timeout + row)
            case BounderUtil.Witnessed => this.copy(witnessed = witnessed + row)
            case BounderUtil.Unreachable => this.copy(unreachable = unreachable + row)
            case BounderUtil.Interrupted(_) => this.copy(err = err + row)
        }
    }
}
case class ApkSummaryRow(apk:String, row:SummaryRow = SummaryRow()) extends TableAble{
    def headers = "apk"::row.headers
    def values = apk::row.values
    def add(newRow:JoinedResultRow):ApkSummaryRow = {
        assert(newRow.apk == apk)
        this.copy(row = row.add(newRow))
    }
}



implicit val getCoffeeResult = GetResult(r => JoinedResultRow(r.<<, r.<<, List(r.<<), r.<<, r.<<, r.<<))
object ReadResults{
//where cast(results.result::json->'summary' as varchar) like '%Witnessed%' or cast(results.result::json->'summary' as varchar) like '%timeout%'
    //TODO:==== figure out if this query is counting things correctly
    def selectResults() = {
        val q = sql"""select results.jobtag, cast(results.result::json->'summary' as varchar), results.loc, results.qry, jobs.config as cfg, 
        cast(jobs.config::json->'apkPath' as varchar) as benchmark from results inner join jobs on jobs.id=results.jobid"""
        Await.result(db.db.run(q.as[JoinedResultRow]), 30 seconds)   
    }
    val allResults = selectResults()
    
    val rand = new scala.util.Random
    
    // Top model
    val allTopResults = allResults.filter(res => read[ExpTag](res.tag).specRefinement == "")
    val topResultsByQuery = allTopResults.groupBy(res => (res.apk,res.qry)).map{ // each query can have multiple locations, we just merge them
        case (_, rows) => rows.reduce((a:JoinedResultRow,b:JoinedResultRow) => a.merge(b))
    }.toSeq
    val topByApp:Seq[ApkSummaryRow] = topResultsByQuery.foldLeft(Map[String,ApkSummaryRow]()){
        case (acc,joinedRow) => 
            acc + (joinedRow.apk -> acc.getOrElse(joinedRow.apk, ApkSummaryRow(joinedRow.apk)).add(joinedRow))    
    }.values.toSeq.sortBy(_.apk)
    val appList = topByApp.map{v => v.apk}
    val topByResult = Seq(topResultsByQuery.foldLeft(SummaryRow()){
        case (acc, joinedRow) => acc.add(joinedRow)
    })
    def summaryForApkTop(app:String) = {
        val res = topByApp.filter(row => row.apk.contains(app))
        assert(res.size == 1, s"Failure, wrong number of results ${res.size}")
        res.head
    }
    def sampleWitForTop(app:String) = {
        val summary = summaryForApkTop(app)
        val witnesses = summary.row.witnessed.toList
        if(witnesses.isEmpty)
            None
        else{
            val max = witnesses.size
            Some(witnesses(rand.nextInt(max)))
        }
    }
    def getTopCfgFor(app:String,clazz:String,method:String,line:Option[Int]) = {
        ???
    }
    
}
println(s"all results: ${ReadResults.allResults.size}")
println(s"all top results: ${ReadResults.allTopResults.size}")
println(s"all top results by qry: ${ReadResults.topResultsByQuery.size}")

all results: 6
all top results: 6
all top results by qry: 6


defined class JoinedResultRow
defined class SummaryRow
defined class ApkSummaryRow
getCoffeeResult: AnyRef with GetResult[JoinedResultRow] = <function1>
defined object ReadResults

In [5]:
ReadResults.topByResult

witnessed,unreachable,timeout,err
3,3,0,0


In [6]:
ReadResults.topByApp

apk,witnessed,unreachable,timeout,err
"""${baseDir}/fdroid/com.gpl.rpg.AndorsTrail/0.8.3/apk/com.gpl.rpg.AndorsTrail_67.apk""",0,1,0,0
"""${baseDir}/fdroid/com.seafile.seadroid2/2.2.48/apk/com.seafile.seadroid2_123.apk""",0,1,0,0
"""${baseDir}/fdroid/de.blinkt.openvpn/0.7.43/apk/de.blinkt.openvpn_198.apk""",1,0,0,0
"""${baseDir}/fdroid/jwtc.android.chess/9.3.10/apk/jwtc.android.chess_192.apk""",1,0,0,0
"""${baseDir}/fdroid/net.justdave.nwsweatheralertswidget/1.1.4/apk/net.justdave.nwsweatheralertswidget_12.apk""",0,1,0,0
"""${baseDir}/fdroid/org.sufficientlysecure.keychain/5.8.2/apk/org.sufficientlysecure.keychain_58902.apk""",1,0,0,0


In [7]:
// TODO: see if we can validate that the same alarms exist
object SamplesToSpecify{
    // choose 1 witness sample per app
    def toInt(s: String): Option[Int] = {
      try {
        Some(s.toInt)
      } catch {
        case e: Exception => None
      }
    }
    def createHeuristicDir(cfg:RunConfig) = {
        val appDirName:String = cfg.outFolder.get.split("/").last
        val appDir = expDir / appDirName
        appDir.createIfNotExists(asDirectory=true)
        val heuristicDir = appDir / cfg.tag.heuristicType
        heuristicDir.createIfNotExists(asDirectory=true)
        heuristicDir
    }
    def writeSamples(count:Int) = {
        // val samples = ReadResults.appList.map(app => ReadResults.sampleWitForTop(app)) //TODO: random choice needs to be moved so it gets more than one per app per run
        val samples = ReadResults.topByApp.flatMap{appRes => 
            Random.shuffle(appRes.row.witnessed).take(count)
        }
        samples.zipWithIndex.foreach{
            case (sample,ind) => 
                val cfg = sample.getConfig
                val heuristicDir = createHeuristicDir(cfg)
                val existingDir = heuristicDir.glob("*").toList.filter(a => !a.name.startsWith("."))
                val existingConfigs = heuristicDir.glob("**/config.json").toList.map(v => read[RunConfig](v.contentAsString))
                val currentCfg = read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))
                if(existingDir.size < count){
                    if(existingConfigs.forall(_ != currentCfg)){
                        val nextNum = if(existingDir.isEmpty) 0 else{ existingDir.flatMap(v => toInt(v.name)).max + 1 }
                        val sampleDir = heuristicDir / nextNum.toString
                        sampleDir.createIfNotExists(asDirectory=true)
                        (sampleDir / "config.json").overwrite(write[RunConfig](currentCfg))
                    }
                }
                
                // val f = expDir / s"sample_${ind}"
                // assert(!f.exists())
                // f.createIfNotExists(asDirectory=true)
                // (f / "apk").overwrite(sample.apk)
                // (f / "summary").overwrite(sample.summary)
                // (f / "configTop.json").overwrite(write[RunConfig](read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))))
        }
    }
}
SamplesToSpecify.writeSamples(1)
println(s"Samples to specify: ${expDir.glob("**/configTop.json").toList.size}")

Samples to specify: 0


defined object SamplesToSpecify

In [8]:
import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}
import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}


import edu.colorado.plv.bounder.ir.{CBEnter, CBExit, CIExit}

object ToSpecify{
    val allCfg = expDir.glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
}
val f = NamedPureVar("f")
val b = NamedPureVar("b")
val l = NamedPureVar("l")
println(ToSpecify.allCfg.size)

37


import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}

import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}



import edu.colorado.plv.bounder.ir.{CBEnter, CBExit, CIExit}


defined object ToSpecify
f: NamedPureVar = NamedPureVar(n = "f")
b: NamedPureVar = NamedPureVar(n = "b")
l: NamedPureVar = NamedPureVar(n = "l")

In [8]:
//ch.blinkenlights.android  (20 min 3 seconds so far) + (42:28) + (9:15) + (8:32) + (3:09)
// note: above pauses were checking other unrelated issues with historia implementation
object Specify_0{
    val initial = read[InitialQuery]("""
        {
          "methodName": "void onClick(android.content.DialogInterface,int)",
          "line": 297,
          "t": "ReceiverNonNull",
          "matcher": "void onBackPressed()",
          "className": "ch.blinkenlights.android.vanilla.PreferencesActivity$HeadsetLaunchFragment$2"
        }
      """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)

    private val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    assert(mConfig.size == 1)
    val config = mConfig.head._2.copy(componentFilter = Some(Seq("ch.blinkenlights.android.vanilla.PreferencesActivity.*",
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$HelpFragment",
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$EqualizerFragment", 
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$PlaylistFragment",
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$ReplayGainFragment", 
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$CoverArtFragment", 
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$NotificationsFragment", 
                                                                 "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$MiscFragment")))
    val configFile = mConfig.head._1
    
    val Fragment_onCreate: SignatureMatcher = SubClassMatcher(SpecSignatures.Fragment,
          "void onCreate\\(android.os.Bundle\\)", "Fragment_onCreate")
                                                                                                             
    val Fragment_onCreate_entry: OAbsMsg = AbsMsg(CBEnter, Fragment_onCreate, TopVal::f::Nil)
    val fragmentActivityNotAttached_improved: LSPred =
        Or(And(
            NS(SpecSignatures.Fragment_onDestroy_exit, SpecSignatures.Fragment_onActivityCreated_entry),
            NS(SpecSignatures.Fragment_onDestroy_exit, Fragment_onCreate_entry)
        ),
          And(Not(SpecSignatures.Fragment_onActivityCreated_entry), Not(Fragment_onCreate_entry)))
    val fragment_resumed:LSPred = NS(Fragment_onCreate_entry, SpecSignatures.Fragment_onDestroy_exit)
    val fragment_create_only_once:LSSpec = LSSpec(f::Nil, Nil, Not(Fragment_onCreate_entry), Fragment_onCreate_entry)
    val DialogBuilder = Set("android.app.AlertDialog$Builder")
    val DialogBuilder_init = AbsMsg(CIExit,SubClassMatcher(DialogBuilder, "android.app.AlertDialog\\$Builder void <init>\\(android.content.Context\\)", "DialogBuilder_init"), TopVal::b::a::Nil)
    val DialogBuilder_set__Button= (negPos:String) => AbsMsg(CIExit, 
                                                         SubClassMatcher(DialogBuilder,
                                                         "android.app.AlertDialog\\$Builder set" + negPos + "Button\\(int,android.content.DialogInterface\\$OnClickListener\\)",
                                                         s"DialogBuilder_set${negPos}Button"), TopVal::b::TopVal::l::Nil)
    val DialogBuilder_setNegativeButton = DialogBuilder_set__Button("Negative")
    val DialogBuilder_setPositiveButton = DialogBuilder_set__Button("Positive")
    val DialogOnClick_onClick = AbsMsg(CBEnter,
                                       SubClassMatcher(Set("android.content.DialogInterface$OnClickListener"),
                                                       "void onClick\\(android.content.DialogInterface,int\\)",
                                                       "DialogOnClick_onClick"), TopVal::l::Nil)
    
    val Dialog_onClick_reg = LSSpec(l::Nil,a::b::f::Nil,
      And(And(And(SpecSignatures.Fragment_get_activity_exit,DialogBuilder_init), fragment_resumed),
      Or(DialogBuilder_setNegativeButton,DialogBuilder_setPositiveButton)), DialogOnClick_onClick)
    
    val configWithSpec = config.copy(specSet = config.specSet.asInstanceOf[PickleSpec]
                                     .copy(specs = 
                                           Set(
                                               fragment_create_only_once,
                                               FragmentGetActivityNullSpec.getActivityNull.copy(pred = fragmentActivityNotAttached_improved),
                                               Dialog_onClick_reg
                                              )))
    def writeSpec():Unit = {
        // verify with added specification
        val outf = configFile.parent / "config_spec.json"
        outf.overwrite(write(configWithSpec))
        
        //check that line is reachable under normal circumstances
        val outfReach = configFile.parent / "config_spec_locreach.json"
        outfReach.overwrite(write(configWithSpec.copy(initialQuery = List(reachableDual))))
    }
}
Specify_0.writeSpec()

cmd8.sc:38: not found: value a
    val DialogBuilder_init = AbsMsg(CIExit,SubClassMatcher(DialogBuilder, "android.app.AlertDialog\\$Builder void <init>\\(android.content.Context\\)", "DialogBuilder_init"), TopVal::b::a::Nil)
                                                                                                                                                                                          ^cmd8.sc:50: not found: value a
    val Dialog_onClick_reg = LSSpec(l::Nil,a::b::f::Nil,
                                           ^Compilation Failed

: 

Compare size of app - callback numbers - callback numbers after manual filtering
By taking the app only perspective we have made the problem harder - what metrics help us tell this story?
- metric: number of callbacks that flowdroid finds versus what app only call graph finds
-

In [ ]:
Specify_0.initial

In [17]:
println(List(1,2,3) zip List('a','b'))


List((1,a), (2,b))
